In [1]:
import os
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '-1' 

/software/spackages/linux-rocky8-x86_64/gcc-9.5.0/anaconda3-2022.05-zyrazrj6uvrtukupqzhaslr63w7hj6in/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Load data
data = np.load('wavelet_transformed_data/layered_452x452_normalized.npy')
data_resized = tf.image.resize(data, [224, 224])
data_resized_np = data_resized.numpy()

# Load labels
labels_df = pd.read_csv('labels/452x452.csv')
labels = labels_df['Label'].values

# Convert labels to binary format
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
binary_labels = to_categorical(integer_encoded)

# Split the data
X_train, X_val, y_train, y_val = train_test_split(data_resized_np, binary_labels, test_size=0.1, random_state=42)

# Fit the ImageDataGenerator
datagen.fit(X_train)

# Load the pretrained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of ResNet50
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using augmented data
model.fit(datagen.flow(X_train, y_train, shuffle=True), steps_per_epoch=len(X_train) // 32, epochs=10, validation_data=(X_val, y_val))


94765736/94765736 [==============================] - 7s 0us/step
Epoch 1/10
17/17 [==============================] - 22s 1s/step - loss: 4.0559 - accuracy: 0.8074 - val_loss: 10.3632 - val_accuracy: 0.1094
Epoch 2/10
17/17 [==============================] - 17s 1s/step - loss: 0.5487 - accuracy: 0.9056 - val_loss: 12.0735 - val_accuracy: 0.1094
Epoch 3/10
17/17 [==============================] - 17s 997ms/step - loss: 0.1621 - accuracy: 0.9389 - val_loss: 10.3714 - val_accuracy: 0.1094
Epoch 4/10
17/17 [==============================] - 18s 1s/step - loss: 0.1172 - accuracy: 0.9426 - val_loss: 8.9289 - val_accuracy: 0.1094
Epoch 5/10
17/17 [==============================] - 17s 1s/step - loss: 0.1007 - accuracy: 0.9537 - val_loss: 5.4414 - val_accuracy: 0.1094
Epoch 6/10
17/17 [==============================] - 17s 1s/step - loss: 0.0918 - accuracy: 0.9651 - val_loss: 5.2136 - val_accuracy: 0.1094
Epoch 7/10
17/17 [==============================] - 17s 1s/step - loss: 0.0857 - accuracy

In [11]:
import os

# Define the directory to save the model
model_dir = 'models'

# Create the directory if it doesn't exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the model
model_save_path = 'models/452x452_trained_resnet_model.h5'

# Save the model in HDF5 format
model.save(model_save_path, save_format='h5')

print(f"Model saved to {model_save_path}")

/home/lnagana/.local/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to models/452x452_trained_resnet_model.h5


In [15]:
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
datagen.fit(X_train)  # Assuming X_train was used to fit the datagen during training
test_data_normalized = datagen.standardize(test_data_resized_np)

# Predict using the model
predictions = model.predict(test_data_normalized[:10])
# Assuming `predicted_classes` contains the predicted class indices from your model
predicted_classes = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_classes)

print(predicted_labels)


1/1 [==============================] - 0s 312ms/step
['Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal'
 'Normal' 'Normal']


In [19]:
individual_element = test_data_resized_np[178]
individual_element_batch = np.expand_dims(individual_element, axis=0)
individual_prediction = model.predict(individual_element_batch)
predicted_class_index = np.argmax(individual_prediction, axis=1) #class index
predicted_label = label_encoder.inverse_transform(predicted_class_index) #label

# Output
print("Predicted Class Index:", predicted_class_index[0])
print("Predicted Label:", predicted_label[0])


1/1 [==============================] - 0s 77ms/step
Predicted Class Index: 1
Predicted Label: Normal
